### Собственная реализация

In [50]:
import sys
sys.path.append('../')
sys.path.append('./')

In [51]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.metrics

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

from tqdm import tqdm

from collections import defaultdict
from collections import OrderedDict

import time
import timeit

import json

from src.utils.plot import plot_acc_and_time
from src.config import DATASET_PATH

from src.models.SVM import CustomSVM
from src.data.split import train_test_split
from src.config import sorted_metrics
from src.config import max_iter, C

%matplotlib inline
sns.set(style='darkgrid', font_scale=1.5)

In [52]:
data = pd.read_csv('../'+DATASET_PATH, index_col=[0, 1])
data = data.reset_index(level=1, drop=True)
data = data.drop(['position', 'age'], axis=1)
data

,x_mean,x_std,x_max,x_min,x_energy,x_iqr,y_mean,y_std,y_max,y_min,...,x_iqr_pos1,y_iqr_pos1,z_iqr_pos1,xy_corr_pos1,xz_corr_pos1,yz_corr_pos1,height,mass,is_valid,sex
_id,,,,,,,,,,,,,,,,,,,,,
2,11156.76,123.532772,11380.0,10836.0,1.244884e+08,177.0,1158.80,73.172275,1348.0,920.0,...,196.0,132.0,485.0,-0.203383,0.750925,-0.452449,185,75,0,1
2,11156.88,128.858961,11380.0,10836.0,1.244924e+08,193.0,1137.96,69.416023,1300.0,920.0,...,196.0,132.0,485.0,-0.203383,0.750925,-0.452449,185,75,0,1
2,11154.36,134.410311,11532.0,10836.0,1.244376e+08,190.0,1088.32,171.829072,1380.0,164.0,...,196.0,132.0,485.0,-0.203383,0.750925,-0.452449,185,75,0,1
2,10998.28,415.506022,11532.0,9384.0,1.211331e+08,323.0,853.36,499.190851,1380.0,-1032.0,...,196.0,132.0,485.0,-0.203383,0.750925,-0.452449,185,75,0,1
2,10883.04,409.575930,11532.0,9384.0,1.186066e+08,543.0,537.68,622.192456,1380.0,-1032.0,...,196.0,132.0,485.0,-0.203383,0.750925,-0.452449,185,75,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,12888.32,56.892399,13028.0,12780.0,1.661120e+08,88.0,707.60,53.817499,856.0,588.0,...,180.0,194.0,220.0,-0.334250,0.872003,-0.319047,176,76,1,1
313,12882.92,56.878691,13028.0,12768.0,1.659728e+08,85.0,699.16,56.771867,856.0,600.0,...,180.0,194.0,220.0,-0.334250,0.872003,-0.319047,176,76,1,1
313,12877.20,55.825124,13028.0,12768.0,1.658254e+08,77.0,672.28,42.218307,804.0,580.0,...,180.0,194.0,220.0,-0.334250,0.872003,-0.319047,176,76,1,1


In [9]:
y = data['is_valid']
X = data.drop('is_valid', axis=1)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

Так как для обучения модели и её хранения нужно малое кол-во памяти, то возьмем 30% от выборки. Это не должно сказаться на точности, так как имеется множество повторяющхися элементов выборки. Также будем использовать float16.\

In [27]:
X_train = X_train[::3]
y_train = y_train[::3]

In [28]:
max_iter = 1000

In [29]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train).astype(np.half)
y_train = y_train.to_numpy(dtype=np.half)[:, np.newaxis]

In [30]:
svc = CustomSVM(max_iter=max_iter)
svc.fit(X_train, y_train)

In [31]:
X_test = scaler.transform(X_test).astype(np.half)
y_test = y_test.to_numpy(dtype=np.half)[:, np.newaxis]

In [32]:
svc.score(X_test, y_test)

0.49224438113327

8 лучших метрик.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X[sorted_metrics[:8]], y)

In [11]:
X_train = X_train[::3]
y_train = y_train[::3]

In [12]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train).astype(np.half)
y_train = y_train.to_numpy(dtype=np.half)

In [13]:
X_test = scaler.transform(X_test).astype(np.half)
y_test = y_test.to_numpy(dtype=np.half)

In [14]:
svc = CustomSVM(max_iter=max_iter)
svc.fit(X_train, y_train[:, np.newaxis])

In [15]:
svc.score(X_test, y_test[:, np.newaxis])

0.765073329712113

In [16]:
y_pred = svc.predict(X_test)

In [17]:
target_names = ['crooked', 'straight']

In [18]:
print(classification_report(y_test.astype(np.int), y_pred.astype(np.int)), target_names)

              precision    recall  f1-score   support

           0       0.91      0.58      0.71      1815
           1       0.70      0.94      0.80      1867

    accuracy                           0.77      3682
   macro avg       0.80      0.76      0.76      3682
weighted avg       0.80      0.77      0.76      3682
 ['crooked', 'straight']


In [45]:
svm_params = {'X_train':svc._X_train.tolist(), 'weights':svc.params_.tolist()}
scaler_params = {'mean':scaler.mean_.tolist(), 'std':scaler.scale_.tolist()}
params = {'svm':svm_params, 'scaler':scaler_params}

In [46]:
from src.config import SVM_WEIGHTS_PATH

ImportError: cannot import name 'SVM_WEIGHTS_PATH' from 'src.config' (../src/config.py)

In [49]:
with open('../Model weights/svm_weights.json', 'w+') as fp:
    json.dump(params, fp)